# AquaRescue - Previsão de Seca por Comunidade

Este notebook realiza uma simulação para prever o risco de seca com base em dados meteorológicos e populacionais das comunidades.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import random

np.random.seed(42)
random.seed(42)

comunidades = ['Comunidade A', 'Comunidade B', 'Comunidade C']
n_dias = 60

dados = []

for comunidade in comunidades:
    base_date = datetime.today() - timedelta(days=n_dias)
    for i in range(n_dias):
        data = base_date + timedelta(days=i)
        temperatura = round(np.random.normal(30, 5), 2)
        umidade = round(np.random.uniform(20, 90), 2)
        ponto_orvalho = round(temperatura - ((100 - umidade) / 5), 2)
        pressao = round(np.random.normal(1013, 10), 2)
        vel_vento = round(np.random.uniform(0, 10), 2)
        dir_vento = random.choice(['N', 'S', 'E', 'W'])
        rajada = vel_vento + np.random.uniform(0, 5)
        radiacao = round(np.random.uniform(200, 1000), 2)
        chuva = round(np.random.exponential(scale=2), 2)
        habitantes = random.randint(100, 1000)
        consumo = round(np.random.uniform(50, 150), 2)
        volume_total = habitantes * consumo
        dados.append([comunidade, data, temperatura, umidade, ponto_orvalho, pressao,
                      vel_vento, dir_vento, rajada, radiacao, chuva, habitantes,
                      consumo, volume_total])

colunas = ['Comunidade', 'Data', 'Temperatura', 'Umidade', 'PontoOrvalho', 'Pressao',
           'VelocidadeVento', 'DirecaoVento', 'RajadaVento', 'Radiacao', 'Chuva',
           'Habitantes', 'ConsumoPorPessoa', 'VolumeTotal']

df = pd.DataFrame(dados, columns=colunas)
df.head()


## Análise de Risco de Seca

In [ ]:

df['RiscoSeca'] = (df['Umidade'] < 30) & (df['Chuva'] < 0.5)

# Criar uma previsão básica
prev = df.groupby('Comunidade').tail(7).groupby('Comunidade')[['Temperatura', 'Umidade', 'Chuva']].mean().reset_index()
prev['VolumePrevisto'] = df.groupby('Comunidade')['VolumeTotal'].mean().values
prev['DataPrevista'] = datetime.today() + timedelta(days=7)
prev


## Visualização do Risco de Seca

In [ ]:

for comunidade in df['Comunidade'].unique():
    sub_df = df[df['Comunidade'] == comunidade]
    plt.figure(figsize=(12,4))
    plt.title(f'Risco de Seca - {comunidade}')
    plt.plot(sub_df['Data'], sub_df['Umidade'], label='Umidade (%)')
    plt.plot(sub_df['Data'], sub_df['Chuva'], label='Chuva (mm)')
    plt.axhline(y=30, color='red', linestyle='--', label='Limite Crítico Umidade')
    plt.axhline(y=0.5, color='orange', linestyle='--', label='Limite Crítico Chuva')
    plt.fill_between(sub_df['Data'], 0, 1, where=sub_df['RiscoSeca'], color='red', alpha=0.1, transform=plt.gca().get_xaxis_transform())
    plt.legend()
    plt.xlabel('Data')
    plt.ylabel('Valor')
    plt.tight_layout()
    plt.show()



## 📊 Exploração de Dados e Justificativa do Modelo

A análise de risco de seca se baseia em variáveis como temperatura, umidade, chuva e volume disponível por pessoa. Para prever o risco de seca, aplicaremos **classificação binária** com `RandomForestClassifier`, pois ele lida bem com dados numéricos, não exige normalização e é robusto a ruídos.

O objetivo é prever se haverá ou não seca (`RiscoSeca`) com base nas variáveis anteriores, permitindo ações antecipadas.



In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Codificar coluna DirecaoVento
df_ml = df.copy()
le = LabelEncoder()
df_ml['DirecaoVento'] = le.fit_transform(df_ml['DirecaoVento'])

# Variáveis para o modelo
X = df_ml[['Temperatura', 'Umidade', 'PontoOrvalho', 'Pressao', 'VelocidadeVento',
           'RajadaVento', 'Radiacao', 'Chuva', 'DirecaoVento']]
y = df_ml['RiscoSeca']

# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:

# Modelo Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Avaliação
y_pred = clf.predict(X_test)
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))


In [ ]:

# Exemplo: prever risco com dados médios de uma comunidade
exemplo = pd.DataFrame([{
    'Temperatura': 34.0,
    'Umidade': 25.0,
    'PontoOrvalho': 20.0,
    'Pressao': 1010,
    'VelocidadeVento': 3.0,
    'RajadaVento': 6.0,
    'Radiacao': 800,
    'Chuva': 0.1,
    'DirecaoVento': le.transform(['N'])[0]
}])

previsao = clf.predict(exemplo)[0]
print("Risco de Seca previsto:", "SIM" if previsao else "NÃO")



## ✅ Conclusão

O modelo Random Forest obteve boa performance na classificação do risco de seca. A escolha se deu por:

- Robustez a dados ruidosos e multivariados
- Boa explicabilidade e importância de features
- Facilidade de uso sem necessidade de normalização

Esse modelo pode ser integrado em um sistema real para prever automaticamente riscos em comunidades com base em dados meteorológicos diários.
